## HW7
### *version one*
(completed using scikit-learn)

In [ ]:
%mkdir data
%cd data
!wget https://www.csie.ntu.edu.tw/~htlin/mooc/datasets/mltech/hw3_dectree_train.dat
!wget https://www.csie.ntu.edu.tw/~htlin/mooc/datasets/mltech/hw3_dectree_test.dat

In [2]:
import numpy as np
import matplotlib.pyplot as plt

trainingdata = np.loadtxt('hw3_dectree_train.dat')
Xtrain = trainingdata[: , :-1]
Ytrain = trainingdata[:,-1]
datanum,featurenum = Xtrain.shape[0],Xtrain.shape[1]
print(datanum,featurenum)

100 2


In [3]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf=clf.fit(X=Xtrain,y=Ytrain)
print(clf.get_n_leaves())

11


In [4]:
Yin = clf.predict(Xtrain)
Ein = np.where(Yin==Ytrain,0,1)
ein = np.sum(Ein)/datanum
print(ein)

0.0


In [5]:
datatest = np.loadtxt('hw3_dectree_test.dat')
Xtest = datatest[:,:-1]
Ytest = datatest[:,-1]
Yout = clf.predict(Xtest)
Eout = np.where(Yout==Ytest,0,1)
print(np.mean(Eout))

0.126


In [ ]:
from sklearn.ensemble import RandomForestClassifier
Ein = []
for i in range(100):
  np.random.seed(i**2)
  clf = RandomForestClassifier(n_estimators=300,bootstrap=True,random_state=np.random.randint(1,1000))
  clf.fit(Xtrain,Ytrain)
  Yout=clf.predict(Xtest)
  Ein.append(np.mean(np.where(Ytest==Yout,0,1)))
print(np.mean(Ein))


0.07343


### *version two*
wheels **:)**



#### decision tree

In [7]:
#----- GINI INDEX------#

def Gini(y):
    if(len(y)==0):
        return 1
    t = np.mean(y==-1)
    return 1 - t**2 - (1-t)**2
# impurty
def lossfunc(theta,X,y):
    """
    theta : decision 阈值集合
    """
    y1 = y[X<theta]
    y2 = y[X>= theta]
    Gini1 = Gini(y1)
    Gini2 = Gini(y2)
    return len(y1)*Gini1 + len(y2)*Gini2

#------decision stump--------#
def Generate_theta(X):
    """ 
    this part is the same as in HW6
    """

    X = np.sort(X)
    theta = (X[1:]+X[:-1]) / 2 #相邻两数均值设为阈值
    theta = np.r_[[X[0]-1],theta]
    theta = np.r_[theta,[X[-1]+1]]

    return theta

def Decision_stump(X,y):
    n,d = X.shape
    Theta = 0
    D=0
    Score = n
    for i in range(d):
        x = X[:,i]
        #获得该列
        theta = Generate_theta(x)
        for theta_ in theta:
            score = lossfunc(theta_,x,y)
            if score < Score:
                Score = score
                Theta = theta_
                D = i
    return D,Theta,Score

In [11]:
#------- learning methods ---------#

def isstop(X,y):
    """ 
    不经修剪的停止条件:x全为一个样本或者y都一样(同类)
    """
    n = X.shape[0]
    n1 = np.sum( y!=y[0] )
    n2 = np.sum(X!=X[0,:])
    return n1==0 or n2==0

class Dtree:
    def __init__(self , theta , dimension , value=None):
        self.theta = theta
        self.dimension = dimension 
        self.value = value        # value corresponde to current node
        self.right = None
        self.left = None
    
def learntree(X,y):
    if isstop(X,y):
        return Dtree(None,None,y[0]),1
    else:
        dimens,theta,score = Decision_stump(X,y)
        tree = Dtree(theta,dimens)
        # data dividing #
        i1 = X[:,dimens] < theta
        X1 = X[i1]
        y1 = y[i1]

        i2 = X[:,dimens] >= theta
        X2 = X[i2]
        y2= y[i2]
        # tree building #
        lefttree ,num1 = learntree(X1,y1)
        righttree , num2 = learntree(X2,y2)
        tree.left = lefttree
        tree.right = righttree
        return tree,num1+num2
    
def pred(tree, x):
    #if tree.left == None and tree.right == None:
    if tree.value != None:
        return tree.value
    if x[tree.dimension] < tree.theta:
        return pred(tree.left, x)
    else:
        return pred(tree.right, x)

def error(tree, X, y):
    ypred = [pred(tree, x) for x in X]
    return np.mean(ypred!=y)

dtree , branches = learntree(Xtrain, Ytrain)

print(error(dtree,Xtrain,Ytrain),branches)


0.0 11


In [12]:
print(error(dtree, Xtest, Ytest))

0.126


#### Random Forest

In [17]:
N = 3000
Ein = np.array([])
tree = []
m, n = trainingdata.shape
for i in range(N):
    index = np.random.randint(0, m, (m))
    X1 = Xtrain[index, :]
    y1 = Ytrain[index]
    dtree,branches = learntree(X1, y1)
    tree.append(dtree)
    Ein = np.append(Ein, error(dtree, Xtrain, Ytrain))

print(np.mean(Ein))

0.052419999999999994


*乐，随机森林已经跑不动一点了:)*